# 환경설정

In [ ]:
import re
import json

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth()  # 처음 실행 시 브라우저 인증
drive = GoogleDrive(gauth)

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
google_dirve_rt_id = os.getenv("GOOGLE_DRIVE_RT_ID")
google_dirve_vtt_id = os.getenv("GOOGLE_DRIVE_VTT_ID")
google_dirve_txt_id = os.getenv("GOOGLE_DRIVE_TXT_ID")

In [ ]:
playlist_id = "PLGiaCgd9PatcGBfZ7xTGdTAsHoNPRQ_AP"

In [ ]:
def upload_to_drive(filename: str, data, google_drive_id: str):
    """ 데이터 구글 드라이브에 저장 """
    file = drive.CreateFile({
        'title': filename,
        'parents': [{'id': google_drive_id}]
    })
    file.SetContentString(data)
    file.Upload()

# 데이터 수집

- 유튜브 플레이 리스트 제목 : 조회수 100만 이상의 즉문즉설 베스트
- 유뷰브 플레이 리스트 ID : `PLGiaCgd9PatcGBfZ7xTGdTAsHoNPRQ_AP`

In [ ]:
# 로컬 저장
# ! yt-dlp --write-auto-sub --sub-lang ko --skip-download -o "data/%(title)s.%(ext)s" "https://www.youtube.com/playlist?list=PLGiaCgd9PatcGBfZ7xTGdTAsHoNPRQ_AP"

In [ ]:
import yt_dlp
import requests

def get_playlist_entries(playlist_url):
    """ 유튜브 플레이리스트 URL 데이터 반환"""
    ydl_opts = {'quiet': True, 'extract_flat': True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(playlist_url, download=False)
        return info['entries']
    
def get_subtitle_text(info_dict):
    """ 유튜브 자막 데이터 반환 """
    # 자막 가져오기
    subtitles = info_dict.get('subtitles') or info_dict.get('automatic_captions')
    
    if not subtitles or 'ko' not in subtitles:
        return None
    
    # 자막 가져오기
    subtitle_url = subtitles['ko'][0]['url']
    response = requests.get(subtitle_url)
    response.encoding = 'utf-8'
    return response.text

In [ ]:
# 유튜브 플레이리스트 URL 정보 로드
playlist_url = f"https://www.youtube.com/playlist?list={playlist_id}"
entries = get_playlist_entries(playlist_url)

In [ ]:
# 자막 추출을 위한 생성자 정의
ydl_opts = {
    'writesubtitles': True,           # 자막 다운로드 활성화
    'skip_download': True,            # 영상 자체는 다운로드하지 않음
    'subtitleslangs': ['ko'],         # 한국어 자막만 대상
    'writeautomaticsub': True,        # 자동 생성 자막(YouTube 자동 자막)도 허용
    'quiet': True,                    # 출력 로그 최소화
    'outtmpl': '-',                   # 파일 저장하지 않음 (stdout 출력 용도)
}
ydl = yt_dlp.YoutubeDL(ydl_opts)

In [ ]:
# 초기화
contents = {}

In [ ]:
from yt_dlp.utils import DownloadError

for entry in entries:
    video_id = entry['id']
    video_url = f"https://www.youtube.com/watch?v={video_id}"

    try:
        info_dict = ydl.extract_info(video_url, download=False)
    except DownloadError  as e:
        print(f"❌ 다운로드 에러 : {video_url}")
        print(f"사유: {e}")
        continue

    title = info_dict.get('title')

    if "몰아보기" in title:
        print(f"❌ 몰아보기 영상 스킵 : {video_url}")
        continue

    text = get_subtitle_text(info_dict)
    if title and text:
        contents[video_id] = {
            'title': title,
            'tags': info_dict.get('tags'),
            'video_url': video_url,
            'view_count': info_dict.get('view_count'),
            'duration': info_dict.get('duration'),
            'like_count': info_dict.get('like_count'),
            'channel': info_dict.get('channel'),
            'upload_date': info_dict.get('upload_date'),
            'subtitles_ko': json.loads(text)
        }
    else:
        print(f"❌ 자막 없음 : {video_url}")

❌ 자막 없음 : https://www.youtube.com/watch?v=zZJ56gguSd4 <br>
❌ 자막 없음 : https://www.youtube.com/watch?v=yIAeCcNxcMU <br>
❌ 자막 없음 : https://www.youtube.com/watch?v=YBi9ycNahBI <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=95yhenTCn2o <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=Vz4SrrF16Xo <br>
❌ 자막 없음 : https://www.youtube.com/watch?v=_IMRG5PhVaM <br>
❌ 자막 없음 : https://www.youtube.com/watch?v=etBucIEnu4U <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=p2uO41Xiw_Q <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=Vc0CVBgINGg <br>
❌ 자막 없음 : https://www.youtube.com/watch?v=aHP4Tkz8NLY <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=nGI5PBZMIoU <br>
❌ 자막 없음 : https://www.youtube.com/watch?v=Gw55vg6KhCY <br>
❌ 자막 없음 : https://www.youtube.com/watch?v=y9DU_JH34N8 <br>
❌ 자막 없음 : https://www.youtube.com/watch?v=eRFVolMIk6E <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=EI6OhhZrVAQ <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=xJlg9C95jS8 <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=wuHJVAlsC8Q <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=bMsksVyI6PM <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=AZe4EDBX47c <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=S36lJ96-V78 <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=votLe7RNhh0 <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=Qut4goomTT0 <br>
❌ 몰아보기 영상 스킵 : https://www.youtube.com/watch?v=sUc5oPA3dRQ <br>
❌ 다운로드 에러 : https://www.youtube.com/watch?v=jybBMfI5ffQ / 사유: ERROR: [youtube] jybBMfI5ffQ: Private video. <br>
❌ 다운로드 에러 : https://www.youtube.com/watch?v=91UmNJ7jmUo / 사유: ERROR: [youtube] 91UmNJ7jmUo: Private video. <br>
❌ 다운로드 에러 : https://www.youtube.com/watch?v=x9XGbwJ0cXo / 사유: ERROR: [youtube] x9XGbwJ0cXo: Private video. <br>
❌ 다운로드 에러 : https://www.youtube.com/watch?v=-NGc_e6S86o / 사유: ERROR: [youtube] -NGc_e6S86o: Private video. <br>
❌ 다운로드 에러 : https://www.youtube.com/watch?v=0Si03LTGDvU / 사유: ERROR: [youtube] 0Si03LTGDvU: Private video. <br>
❌ 다운로드 에러 : https://www.youtube.com/watch?v=reWBasfXgJ4 / 사유: ERROR: [youtube] reWBasfXgJ4: Private video. <br>
❌ 다운로드 에러 : https://www.youtube.com/watch?v=6bAgrJ-3r5k / 사유: ERROR: [youtube] 6bAgrJ-3r5k: Video unavailable. <br>
❌ 다운로드 에러 : https://www.youtube.com/watch?v=0-FcVgomFjM / 사유: ERROR: [youtube] 0-FcVgomFjM: Private video. <br>
❌ 다운로드 에러 : https://www.youtube.com/watch?v=WhmFsQQOtsk / 사유: ERROR: [youtube] WhmFsQQOtsk: Private video.

In [ ]:
# 데이터 저장
upload_to_drive(
    f'{playlist_id}.json', 
    json.dumps(contents, ensure_ascii=False, indent=2), 
    google_dirve_vtt_id
)

# 데이터 가공

### 1차 가공 (vtt → txt)

In [ ]:
def clean_content(title:str, content:dict) -> str:
    """ 자막 데이터 1차 가공 """
    if "events" not in content:
        print(title, '..Empty..')
        print('==='*20)
        return None

    segs = []
    for event in content['events']:
        if "segs" not in event:
            continue

        segs += event["segs"]
    
    cleaned_content = ' '.join([seg["utf8"] for seg in segs])
    cleaned_content = cleaned_content.replace('[박수]', ' ')
    cleaned_content = cleaned_content.replace('[웃음]', ' ')
    cleaned_content = cleaned_content.replace('[음악]', ' ')
    cleaned_content = cleaned_content.replace('(청중 웃음)', ' ')
    cleaned_content = cleaned_content.replace('(청중 박수)', ' ')
    cleaned_content = re.sub(r'\s+', ' ', cleaned_content).strip()

    print(title)
    print('---'*20)
    print(f'{cleaned_content[:10]}...[{len(cleaned_content)}]', )
    print('==='*20)
    return cleaned_content

In [ ]:
# 파일 탐색
file = drive.ListFile({
    'q': f"'{google_dirve_vtt_id}' in parents and title = '{playlist_id}.json' and trashed = false"
}).GetList()

# 파일 로드
if file:
    original_contents = json.loads(file[0].GetContentString())

len(original_contents), type(original_contents)

In [ ]:
# 자막 데이터 1차 가공
cleaned_contents = {}
for video_id, values in original_contents.items():
    title = values.get('title')
    content = values.get('subtitles_ko')
    text = clean_content(title, content) # 자막 데이터 1차 가공
    if text:
        cleaned_contents[video_id] = values
        cleaned_contents[video_id]['subtitles_ko'] = text

In [ ]:
# 데이터 저장
upload_to_drive(
    f'{playlist_id}.json', 
    json.dumps(cleaned_contents, ensure_ascii=False, indent=2), 
    google_dirve_txt_id
)

### 2차 가공 (txt → json)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1
)

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain_core.output_parsers import StrOutputParser

# 자막 요약 체인 구성
system_template = """당신은 법륜스님의 즉문즉설 자막을 요약하는 AI입니다. 
당신의 작업은 다음과 같습니다:

1. 자막 전체를 읽고 누가 말했는지 판단합니다. 입력으로 주어지는 자막은 줄글이며, 화자가 명시되지 않은 경우도 있습니다. 자막에는 가끔 (스님), (질문자) 같은 표기가 포함되어 있는데, 이 표기는 다음 단어의 화자를 의미합니다. 예: (질문자) 네 → 질문자가 "네"라고 말한 것  / (스님) 괜찮아 → 스님이 "괜찮아"라고 말한 것
2. 대화 흐름을 유지한 채로 "스님:"과 "질문자:" 형식으로 대사를 나눕니다.  
3. 말의 핵심 내용만 남겨 **간결하게 요약**하되, 대화의 흐름과 화자의 말투는 유지합니다. 말투와 분위기는 자연스럽게 유지해 주세요. 경전 이름, 일화, 농담, 강조가 있으면 표현을 그대로 살려 주세요.
4. 생략 없이, 말한 사람과 순서를 정확히 반영해 주세요.

결과는 아래 형식으로 출력해 주세요:

질문자: (요약된 질문)  
스님: (요약된 답변)  
질문자: (요약된 중간 반응 또는 후속 질문)  
스님: (요약된 답변)
"""
system_message = SystemMessagePromptTemplate.from_template(template=system_template)

human_template = """다음은 즉문즉설 대화 내용입니다. 이 내용을 대화 형식을 유지하면서 요약해 주세요.

[자막 원문]
{content}

[자막 요약]
"""
human_message = HumanMessagePromptTemplate.from_template(template=human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
chain = chat_prompt | llm | StrOutputParser()

In [ ]:
# 파일 탐색
file = drive.ListFile({
    'q': f"'{google_dirve_txt_id}' in parents and title = '{playlist_id}.json' and trashed = false"
}).GetList()

# 파일 로드
if file:
    original_contents = json.loads(file[0].GetContentString())

len(original_contents), type(original_contents)

In [ ]:
# 초기화
contents = {}

In [ ]:
for key, vals in original_contents.items():
    title = vals.get('title')
    content = vals.get('subtitles_ko')
    summary_content = chain.invoke({"content": content})    # 자막 데이터 2차 가공

    if summary_content:
        values = vals.copy()
        values.pop('subtitles_ko')
        
        values['content'] = summary_content
        contents[key] = values

        print(title)
        print('---'*10)
        print(summary_content[:100])
        print('==='*10)

In [ ]:
from pprint import pprint

pprint(summary_content)

In [ ]:
# 데이터 저장
upload_to_drive(
    f'{playlist_id}.json', 
    json.dumps(contents, ensure_ascii=False, indent=2), 
    google_dirve_rt_id
)